In [1]:
%load_ext autoreload
%autoreload 2

from pandas_datareader import data as wb
import os
import pandas as pd

stocks = pd.read_csv('data/stocks.csv').fillna(0)
stocks = stocks.set_index('Unnamed: 0')

for s in stocks:
    d = wb.DataReader(s, data_source='yahoo', start='1997-1-1')
    file_path = os.path.join('data/price', s+'.csv')
    d.to_csv(file_path)

In [2]:
# Manual input
usd_to_sgd_rate = 1.3238            # USD -> SGD
remaining_usd_cash = 10534.43+57.84  # Remaining USD cash in IB and DBS

In [3]:
def load_data(tick):
    file_path = os.path.join('data/price', tick+'.csv')
    data = pd.read_csv(file_path)
    dt_idx = pd.DatetimeIndex(data['Date'])
    data.index = dt_idx

    return data

def get_transactions():
    stock_buy_transactions = pd.read_csv('data/stock_buy_transactions.csv')

    stock_buy_transactions['total_cost'] = stock_buy_transactions['buy_price']*stock_buy_transactions['quantity']\
        +stock_buy_transactions['commission']
    return stock_buy_transactions

def get_positions():
    stock_buy_transactions = get_transactions()
    df = pd.DataFrame()
    df['tick'] = stock_buy_transactions['tick'].drop_duplicates()
    
    stock_dividends = pd.read_csv('data/stock_dividends.csv')
    df = pd.merge(df, stock_dividends.groupby(['tick']).sum(), on='tick', how='outer')
    df = df.rename(columns={'amount':'dividends'})
    df['dividends'] = df['dividends'].fillna(0)
    df = pd.merge(df, stock_buy_transactions.groupby(['tick']).sum(), on='tick', how='outer')
    df['avg_cost'] = df['total_cost']/df['quantity']
    df = df.drop(columns=['buy_price'])

    df['currency_sgd'] = 'SGD'
    df['current_price'] = 0.0
    for index, row in df.iterrows():
        t = row['tick']
        d = load_data(t)
        current_price = d['Adj Close'].iloc[-1]
        df.at[index, 'current_price'] = current_price
        df.at[index, 'currency_sgd'] = 1
        if stocks[t]['currency']=='USD':
            df.at[index, 'currency_sgd'] = usd_to_sgd_rate

    df['pl'] = df['current_price']*df['quantity']+df['dividends']-df['total_cost']
    df['return'] = (df['current_price']*df['quantity']+df['dividends'])/df['total_cost']-1.0
    df['current_value_sgd'] = df['current_price']*df['quantity']*df['currency_sgd']
    df['allocation'] = df['current_value_sgd']/df['current_value_sgd'].sum()

    return df

def get_positions_vis(df):
    df_vis = pd.DataFrame()
    df_vis['tick'] = df['tick']
    df_vis['dividends'] = df['dividends']
    df_vis['quantity'] = df['quantity']
    df_vis['commission'] = df['commission']
    df_vis['total_cost'] = df['total_cost'].map('{:,.2f}'.format)
    df_vis['avg_cost'] = df['avg_cost'].map('{:,.2f}'.format)
    df_vis['current_price'] = df['current_price'].map('{:,.2f}'.format)
    df_vis['pl'] = df['pl'].map('{:,.2f}'.format)
    df_vis['return'] = df['return'].map('{:,.2%}'.format)
    df_vis['current_value_sgd'] = df['current_value_sgd']
    df_vis['allocation'] = df['allocation'].map('{:,.2%}'.format)
    df=df.set_index('tick')
    df_vis=df_vis.set_index('tick')

    return df_vis

df_positions = get_positions()
df_vis = get_positions_vis(df_positions)
df_vis

,dividends,quantity,commission,total_cost,avg_cost,current_price,pl,return,current_value_sgd,allocation
tick,,,,,,,,,,
D05.SI,252.00,700,24.61,"14,304.61",20.44,29.71,"6,744.39",47.15%,20797,6.46%
ME8U.SI,286.80,5000,33.18,"15,253.18",3.05,2.77,"-1,116.38",-7.32%,13850,4.30%
AJBU.SI,134.26,3500,16.70,"10,026.70",2.86,2.59,-827.44,-8.25%,9065,2.82%
C2PU.SI,213.60,2000,14.25,"8,334.25",4.17,4.31,499.35,5.99%,8620,2.68%
BABA,-1.87,91,77.04,"22,396.82",246.12,219.02,"-2,467.87",-11.02%,26384.4,8.20%
AAPL,9.24,62,19.26,"7,081.68",114.22,125.89,732.74,10.35%,10332.5,3.21%
MSFT,9.05,34,19.26,"7,101.80",208.88,250.79,"1,434.11",20.19%,11287.9,3.51%
NVDA,0.00,18,19.26,"9,636.12",535.34,703.13,"3,020.22",31.34%,16754.5,5.21%
ARKK,108.14,95,20.26,"9,869.46",103.89,109.51,642.13,6.51%,13772.1,4.28%


In [4]:
def rough_pl_summary(df):
    stock_others = pd.read_csv('data/stock_others.csv') # TODO: not calculated yet

    print('SG stocks PL:')
    print(df[df['currency_sgd']==1]['pl'].sum())
    print('\nUS stocks PL (in SGD):')
    print(df[df['currency_sgd']!=1]['pl'].sum()*usd_to_sgd_rate)
    print('\nTotal PL (in SGD)')
    print(df[df['currency_sgd']==1]['pl'].sum()+df[df['currency_sgd']!=1]['pl'].sum()*usd_to_sgd_rate)

rough_pl_summary(df_positions)

SG stocks PL:
5299.9188489151

US stocks PL (in SGD):
6470.903148059944

Total PL (in SGD)
11770.821996975044


In [5]:
def acutual_pl_summary(df_pos):
    from datetime import datetime, timedelta

    df_cur_tran = pd.read_csv('data/currency_exchange_transactions.csv')
    total_original_sgd_to_usd = df_cur_tran[df_cur_tran.currency=='SGDUSD']['amount'].sum()
    print('Total Converted: SGD -> USD\t' + str(total_original_sgd_to_usd))
    df_cur_tran['coverted_amount'] = df_cur_tran['amount'] / df_cur_tran['rate']
    coverted_usd = df_cur_tran.loc[df_cur_tran.currency=='SGDUSD']['coverted_amount'].sum()
    print('Total Converted USD\t\t' + str(coverted_usd))
    print('Avg USD to SGD rate \t\t' + '{:.4}'.format(total_original_sgd_to_usd/coverted_usd))
    current_value_usd = df_pos[df_pos['currency_sgd']!=1]['current_value_sgd'].sum()/usd_to_sgd_rate
    print('Positions Current Value USD\t' + str(current_value_usd))    
    usd_total_remain_sgd = df_pos[df_pos['currency_sgd']!=1]['current_value_sgd'].sum()+remaining_usd_cash*usd_to_sgd_rate
    print('USD NAV Value USD\t\t' + str(usd_total_remain_sgd))
    
    print('\n')
    
    total_sgd_cost = df_pos[df_pos['currency_sgd']==1]['total_cost'].sum()
    print('Total SGD Stocks Cost\t\t' + str(total_sgd_cost) + '\n')
    sgd_pl = df_pos[df_pos['currency_sgd']==1]['pl'].sum()
    now = datetime.now()
    
    stock_buy_transactions = get_transactions()
    stock_buy_transactions['expense_ratio']=0.0
    stock_buy_transactions['days'] = stock_buy_transactions['time'].apply(\
                                                                lambda x: (now-datetime.strptime(x, '%d/%m/%y')).days)
    for index, row in stock_buy_transactions.iterrows():
        stock_buy_transactions.at[index, 'expense_ratio'] = stocks[row['tick']]['expense_ratio']
    stock_buy_transactions['expense']=stock_buy_transactions['days']/365*stock_buy_transactions['expense_ratio']\
                                        *stock_buy_transactions['total_cost']/100*usd_to_sgd_rate
    total_expense = stock_buy_transactions['expense'].sum()
    actual_pl = usd_total_remain_sgd-total_original_sgd_to_usd+sgd_pl-total_expense

    print('Potential Expense:\t\t'+ str(total_expense))

    print('Actual Total PL (in SGD):\t' + str(actual_pl))
    print('Actual Total Return\t\t' + '{:.2%}'.format(actual_pl/(total_original_sgd_to_usd+total_sgd_cost)))

    stock_buy_transactions.loc[(stock_buy_transactions.currency == 'SGD'),'currency']=1.0
    stock_buy_transactions.loc[(stock_buy_transactions.currency == 'USD'),'currency']=usd_to_sgd_rate
    stock_buy_transactions['annualized_cost_sgd'] = stock_buy_transactions['currency']*stock_buy_transactions['days']\
                                                    /365*stock_buy_transactions['total_cost']
    annualized_cost_sgd = stock_buy_transactions['annualized_cost_sgd'].sum()
    print('Annulized Invest Time\t\t' + '{:.2%}'.format(annualized_cost_sgd/(total_original_sgd_to_usd+total_sgd_cost)))
    print('Annulized Return \t\t' + '{:.2%}'.format(actual_pl/annualized_cost_sgd))
    
    return df_cur_tran

df_cur_tran = acutual_pl_summary(df_positions)

Total Converted: SGD -> USD	283472.44175
Total Converted USD		210463.94802134854
Avg USD to SGD rate 		1.347
Positions Current Value USD	203525.00147247312
USD NAV Value USD		283448.4439752599


Total SGD Stocks Cost		47918.740000000005

Potential Expense:		341.54801508514964
Actual Total PL (in SGD):	4934.373059089851
Actual Total Return		1.49%
Annulized Invest Time		42.19%
Annulized Return 		3.53%


In [7]:
def ib_checksum(df_cur_tran):
    df_ib_tran = pd.read_csv('data/transfer_to_ib_transactions.csv')

    total_transfered_sgd = df_ib_tran[df_ib_tran.currency=='SGD']['amount'].sum()
    sgd_coverted = df_cur_tran.loc[df_cur_tran.place=='ib']['amount'].sum()

    stock_buy_transactions = get_transactions()
    stock_sgd_purchase = stock_buy_transactions.loc[(stock_buy_transactions.currency=='SGD') & \
                                (stock_buy_transactions['commission']<5)]['total_cost'].sum()

    print(total_transfered_sgd-sgd_coverted-stock_sgd_purchase) # this number should be very close
    
ib_checksum(df_cur_tran)

1.0582499999873107


In [13]:
def position_industry(df_pos):
    stock_profile = pd.read_csv('data/stock_profile.csv').fillna(0)
    stock_profile = stock_profile.set_index('Unnamed: 0')
    df = df_pos.set_index("tick")
    stock_profile['allocation'] = df['allocation']

    for column in stock_profile:
        if column == 'allocation':
            continue
        stock_profile[column] = stock_profile[column]*df['allocation']

    
    stock_profile.loc["Total"] = stock_profile.sum()
    default_float_format = pd.options.display.float_format
    pd.options.display.float_format = '{:.2%}'.format

    display(stock_profile)

    pd.options.display.float_format = default_float_format

position_industry(df_positions)

,Basic Materials,Consumer Cyclical,Financial Services,Real Estate,Consumer Defensive,Healthcare,Utilities,Communication Services,Energy,Industrials,Technology,allocation
Unnamed: 0,,,,,,,,,,,,
SOXX,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,22.29%,22.29%
CQQQ,0.00%,0.16%,0.00%,0.00%,0.00%,0.00%,0.00%,0.67%,0.00%,0.02%,0.82%,1.67%
QQQ,0.00%,0.20%,0.03%,0.00%,0.05%,0.07%,0.01%,0.22%,0.00%,0.03%,0.49%,1.10%
SPY,0.09%,0.48%,0.55%,0.10%,0.24%,0.49%,0.10%,0.43%,0.10%,0.35%,0.90%,3.83%
ARKK,0.00%,0.50%,0.09%,0.06%,0.05%,1.25%,0.00%,1.26%,0.00%,0.09%,0.98%,4.28%
ARKG,0.00%,0.00%,0.02%,0.00%,0.02%,4.12%,0.00%,0.00%,0.00%,0.00%,0.07%,4.23%
ARKW,0.00%,0.48%,0.19%,0.08%,0.02%,0.15%,0.00%,0.81%,0.00%,0.00%,0.82%,2.57%
ARKQ,0.00%,0.78%,0.08%,0.00%,0.04%,0.04%,0.00%,0.45%,0.00%,0.81%,1.03%,3.23%
ARKF,0.00%,0.28%,0.42%,0.07%,0.00%,0.05%,0.00%,0.47%,0.00%,0.00%,0.67%,1.95%
